In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [5]:

df = pd.read_excel('heart_short.xlsx',sheet_name="data")
df


,age,t_index,f_status,c_index,target
0,63,145,1,233,1
1,37,130,0,250,1
2,41,130,0,204,1
3,56,120,0,236,1
4,57,120,0,354,1
5,57,140,0,192,1
6,56,140,0,294,1
7,44,120,0,263,1
8,52,172,1,199,1
9,57,150,0,168,1


In [ ]:
# xác định coll input, output
X = df[['age','t_index','f_status','c_index']].values # input
y = df[['target']].values # output

# ép kiểu về float
X = X.astype(float)
y = y.astype(float)


In [6]:
# spliting data set: 14 mẫu train và 6 mẫu test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=16)

In [7]:
# build model K-NN with 5k
from sklearn.neighbors import KNeighborsClassifier
knnModel = KNeighborsClassifier(n_neighbors=5)

In [8]:
# training model
knnModel.fit(X_train, y_train)

C:\Users\datba\AppData\Roaming\Python\Python311\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [9]:
# Dự báo mô hình trên tâp test
y_test_predicted = knnModel.predict(X_test)
y_test_predicted


array([0., 0., 0., 0., 0., 0.])

In [11]:

knnModel.score(X_test, y_test)


0.6666666666666666

In [12]:

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_test_predicted)

from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, y_test_predicted)

#Ngang là dọc, dọc là ngang, cẩn thận khi đọc
cf_matrix



array([[4, 0],
       [2, 0]])

<img src='./img/image.png'>

In [13]:

def myscores(smat): 
    tp = smat[1][1] 
    fp = smat[0][1] 
    fn = smat[0][0] 
    tn = smat[0][1] 
    vprecision = tp/(tp+fp)
    vrecall = tp/(tp+fn)
    vf1 = 2*(vprecision*vrecall)/(vprecision+vrecall) 

    return vprecision,vrecall,vf1
print("precision, recall, f1:", myscores(cf_matrix))


precision, recall, f1: (np.float64(0.0), np.float64(0.0), np.float64(nan))


C:\Users\datba\AppData\Local\Temp\ipykernel_38792\1128620245.py:8: RuntimeWarning: invalid value encountered in scalar divide
  vf1 = 2*(vprecision*vrecall)/(vprecision+vrecall)


In [14]:

from sklearn.metrics import classification_report
target_names = ['Không bệnh', 'Có bệnh']
print(classification_report(y_test, y_test_predicted, target_names=target_names))


              precision    recall  f1-score   support

  Không bệnh       0.67      1.00      0.80         4
     Có bệnh       0.00      0.00      0.00         2

    accuracy                           0.67         6
   macro avg       0.33      0.50      0.40         6
weighted avg       0.44      0.67      0.53         6



C:\Users\datba\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\datba\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\datba\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [ ]:

from sklearn import metrics

y_pred_proba = knnModel.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,'go-',label="AUC="+str(auc))
plt.plot([0,1],[0,1],'r--')
plt.title("AUC & ROC Curve")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc=4)
plt.fill_between(fpr, tpr, facecolor='lightgreen', alpha=0.7)
plt.show()


In [15]:
# Xây dựng mô hình dự báo
import pickle
pickle.dump(knnModel, open('model_KNN_Heart.sav', 'wb'))


In [16]:

import pickle
#Load model từ storage
loaded_model = pickle.load(open('model_KNN_Heart.sav', 'rb'))


In [18]:

v1 = float(input('age: '))
v2 = float(input('t_index: '))
v3 = float(input('f_status: '))
v4 = float(input('c_index: '))

y_pred = loaded_model.predict([[v1,v2,v3,v4]])
print('Kết quả dự báo bệnh tim: '+ str(y_pred[0]))

if (y_pred[0] == 1):
    print("Bị bệnh tim")
else:
    print("Không bị bệnh")


Kết quả dự báo bệnh tim: 1.0
Bị bệnh tim
